Part 1

In [369]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [370]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text 
soup = BeautifulSoup(data, 'html5lib')

In [371]:
soup.prettify()

'<!DOCTYPE html>\n<html class="client-nojs" dir="ltr" lang="en">\n <head>\n  <meta charset="utf-8"/>\n  <title>\n   List of postal codes of Canada: M - Wikipedia\n  </title>\n  <script>\n   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"c4ca0242-ecf7-48bf-8972-2e39787f41fa","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1013111980,"wgRevisionId":1013111980,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","

In [454]:
postal = []
borough = []
borough1 = []
borough2 = []
neighbour = []
neighbour1= []
table = soup.find('table')
for row in table.find_all('tr'): 
    cols = row.find_all('td')
    for columns in cols:
        postal.append(columns.b.string)
        borough.append([link.get('title') for link in columns.find_all('a')])

In [455]:

for row in borough:
    borough1.extend(row[0:1])
for row in borough1:
    borough2.extend(str(row).split('(')[0:1])
for row in borough:
    neighbour.append(row[1:])
for row in neighbour:
    neighbour1.append(str(row).replace(', Toronto','').replace('[\'','').replace('\']','').replace('\'',''))


In [456]:
neighbourhood_data = pd.DataFrame(list(zip(postal,borough2, neighbour1)),columns=['PostalCode','Borough', 'Neighbourhood'])
neighbourhood_data['Borough'] = neighbourhood_data['Borough'].apply(lambda x: ''.join(map(str, x)))
neighbourhood_data['Neighbourhood'] = neighbourhood_data['Neighbourhood'].apply(lambda x: ''.join(map(str,x)))
neighbourhood_data.head(12)

,PostalCode,Borough,Neighbourhood
0,M1A,North York,[]
1,M2A,North York,[]
2,M3A,Downtown Toronto,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Queen's Park,"Regent Park, Harbourfront"
5,M6A,Etobicoke,"Lawrence Manor, Lawrence Heights"
6,M7A,"Scarborough, Toronto",[]
7,M8A,North York,[]
8,M9A,East York,Islington Avenue
9,M1B,Downtown Toronto,"Malvern, Rouge"


In [482]:
neighbourhood_data['Neighbourhood'].replace('[]',np.nan,inplace=True)
neighbourhood_data['Borough'].replace('[]',np.nan,inplace=True)

In [483]:
neighbourhood_data

,level_0,index,PostalCode,Borough,Neighbourhood
0,0,2,M3A,Downtown Toronto,Parkwoods
1,1,3,M4A,North York,Victoria Village
2,2,4,M5A,Queen's Park,"Regent Park, Harbourfront"
3,3,5,M6A,Etobicoke,"Lawrence Manor, Lawrence Heights"
4,4,8,M9A,East York,Islington Avenue
5,5,9,M1B,Downtown Toronto,"Malvern, Rouge"
6,6,11,M3B,Etobicoke,Don Mills
7,7,12,M4B,"Scarborough, Toronto","Parkview Hill, Woodbine Gardens"
8,8,13,M5B,North York,"Garden District, Ryerson University"
9,9,17,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov..."


In [487]:
neighbourhood_data.dropna(inplace=True)
neighbourhood_data.drop(columns=['level_0','index'],inplace=True)

In [489]:
neighbourhood_data.shape

(59, 3)